In [1]:
from importlib import reload
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")


In [2]:
import pandas as pd
from numpy.linalg import lstsq
import utils
from triangle import Triangle
from glm import Poisson
import plotly.express as px
import plotly.graph_objects as go

from scipy.stats import probplot
import scipy

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_poisson_deviance, median_absolute_error, d2_tweedie_score
from sklearn.preprocessing import add_dummy_feature
from sklearn.model_selection import train_test_split, LeaveOneOut, TimeSeriesSplit, cross_val_predict, cross_val_score

import statsmodels as sm
import statsmodels.graphics.gofplots as smgof
import plotly.graph_objs as go
from scipy.stats import norm


%matplotlib inline

In [3]:
def load_tri(type, d, file, rng) -> Triangle:
    t = tr.Triangle.from_excel(file, id=type, origin_columns=1, sheet_name=d[type], sheet_range=rng)
    return(t)

In [4]:
import chainladder
chainladder = reload(chainladder)
import triangle as tr
tr = reload(tr)
utils = reload(utils)


file = r"C:\Users\AndyW\OneDrive\work\2022Q4 - OL Occ.xlsx"

ids = ['Reported Loss', 'Paid Loss', 'Paid DCCE', 'Reported Count', 'Closed Count']
sheets = ['Reported Loss Development', 'Paid Loss Development', "Paid DCCE Development", "Reported Count Development", "Closed Count Development"]
sht_dict = dict(zip(ids, sheets))
cum_triangle_rng = "B5:CD25"


rpt_loss = load_tri('Reported Loss', sht_dict, file, cum_triangle_rng)
paid_loss = load_tri('Paid Loss', sht_dict, file, cum_triangle_rng)
paid_dcce = load_tri('Paid DCCE', sht_dict, file, cum_triangle_rng)
rpt_count = load_tri('Reported Count', sht_dict, file, cum_triangle_rng)
closed_count = load_tri('Closed Count', sht_dict, file, cum_triangle_rng)

# rpt_loss.tri.ata('simple')

In [9]:
    ye_rpt_loss_df = rpt_loss.tri.loc[:, rpt_loss.tri.columns.to_series().mod(12).eq(0)]
    ye_paid_loss_df = paid_loss.tri.loc[:, paid_loss.tri.columns.to_series().mod(12).eq(0)]

    ye_rpt_count_df = rpt_count.tri.loc[:, rpt_count.tri.columns.to_series().mod(12).eq(0)]
    ye_closed_count_df = closed_count.tri.loc[:, closed_count.tri.columns.to_series().mod(12).eq(0)]

In [7]:
import rocky as r
r = reload(r)

#### create a rocky3 object

In [9]:
rsv = r.ROCKY()

#### check the rocky object

In [10]:
rsv

ROCKY(id=None, model=(), tri=(), forecast=(), validate=())

the `ROCKY`  object is created with the following blank attributes:
1. `id` - the id of the rocky object and how it is accessed in rocky3
2. `model` - container for fitting models to the data. any number of models can be fit.
3. `tri` - container for the loss triangles used as input to the models. multiple triangles can be used, and are distinguished by their own `id` attributes.
4. `forecast` - container for the forecast and future trend scenarios. multiple scenarios can be used, and are distinguished by their own `id` attributes.
5. `validate` - this does not really store anything, but is the accessor for the validation functionality.
6. `plot` - this does not really store anything, but is the accessor for the plotting functionality.


since we don't have any triangles (eg `tri=()`), first we load a sample triangle from the `rocky` package. this is the taylor-ashe dataset, which has been used in reserving papers for decades. it is a simple triangle with 10 years of development and 10 accident years.

In [11]:
# add taylor-ashe triangle:
rsv.tri_from_csv(id='paid_loss', file_path="./data/taylorashe.csv", origin_columns=1)

# check the triangle:
rsv.tri

"paid_loss"

note that we access the triangle in our reserving analysis with `rsv.tri.paid_loss`:
* `rsv` is the name of our `ROCKY3` object
* `tri` is the name of the triangle container, and is used to access the triangles in our analysis
* `paid_loss` is the `id` attribute given to the triangle when it was loaded. this is the name we use to access the triangle in our analysis.

so the formula for accessing the triangle is `(analysis object "rsv").(triangle accessor "tri").(triangle id "paid_loss")`, which is `rsv.tri.paid_loss`.

many objects that are part of a `rocky3` model are stored this way: `(analysis object).(object accessor).(object id)`. this is the same for the `model`, `forecast`, and `validate` containers. the `plot` container is a little different, but we'll get to that later.

in general if you can set up a set of assumptions for something, it is containerized in `rocky3`.


now that we have a triangle, we should exlore it a little. a rocky3 `Triangle` object has a number of attributes that are useful for exploring the triangle:  

In [12]:
# display a triangle with age-to-age factors:
rsv.tri.paid_loss.ata()

,1,2,3,4,5,6,7,8,9,10
AY,,,,,,,,,,
1,3.143200,1.542806,1.278299,1.237719,1.209207,1.044079,1.040374,1.063009,1.017725,NaN
2,3.510582,1.755493,1.545286,1.132926,1.084493,1.128106,1.057268,1.086496,NaN,NaN
3,4.448450,1.716718,1.458257,1.232079,1.036860,1.120010,1.060577,NaN,NaN,NaN
4,4.568002,1.547052,1.711784,1.072518,1.087360,1.047076,NaN,NaN,NaN,NaN
5,2.564198,1.872956,1.361545,1.174217,1.138315,NaN,NaN,NaN,NaN,NaN
6,3.365588,1.635679,1.369162,1.236443,NaN,NaN,NaN,NaN,NaN,NaN
7,2.922798,1.878099,1.439393,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3.953288,2.015651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.619179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


note again the formula for accessing the triangle is `(analysis object).(triangle accessor).(triangle id)`, which is `rsv.tri.paid_loss`. to build an age-to-age factor triangle, call the triangle you want, then the `.ata()` method with no arguments gives you the age-to-age factor triangle.

adding in areguments to the `.ata()` method transforms the triangle into a set of average ata factors. available args to change the ata factor type are initially:
1. `'vwa'` - all years volume weighted average
2. `'simple'` - all years simple average
3. `'medial'` - all years medial average, excluding the highest and lowest values

we can make a table with each of these:

In [13]:
ata_fct_df = pd.DataFrame(dict(vwa=rsv.tri.paid_loss.ata('vwa'),
                               simple=rsv.tri.paid_loss.ata('simple'),
                               medial=rsv.tri.paid_loss.ata('medial')))

ata_fct_df.transpose()

,1,2,3,4,5,6,7,8,9,10
vwa,3.490607,1.747333,1.457413,1.173852,1.103824,1.086269,1.053874,1.076555,1.017725,1.0
simple,3.566143,1.745557,1.451961,1.180984,1.111247,1.084818,1.052739,1.074753,1.017725,1.0
medial,3.566155,1.734333,1.434728,1.193916,1.103389,1.083543,1.057268,1.076555,1.017725,1.0


we can add more arguments to the `.ata()` method to change the number of years in the average, or whether just the highest, just the lowest, or both the highest and lowest years are excluded from the medial average.

In [14]:
rsv.tri.paid_loss.ata()

,1,2,3,4,5,6,7,8,9,10
AY,,,,,,,,,,
1,3.143200,1.542806,1.278299,1.237719,1.209207,1.044079,1.040374,1.063009,1.017725,NaN
2,3.510582,1.755493,1.545286,1.132926,1.084493,1.128106,1.057268,1.086496,NaN,NaN
3,4.448450,1.716718,1.458257,1.232079,1.036860,1.120010,1.060577,NaN,NaN,NaN
4,4.568002,1.547052,1.711784,1.072518,1.087360,1.047076,NaN,NaN,NaN,NaN
5,2.564198,1.872956,1.361545,1.174217,1.138315,NaN,NaN,NaN,NaN,NaN
6,3.365588,1.635679,1.369162,1.236443,NaN,NaN,NaN,NaN,NaN,NaN
7,2.922798,1.878099,1.439393,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3.953288,2.015651,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,3.619179,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
t1 = rsv.tri.paid_loss
type(t1)

triangle.Triangle

In [16]:
print(t1.id)

paid_loss


In [17]:
(t1.tri
 .reset_index()
 .rename(columns={'index':'origin'})
#  .melt(id_vars='origin', var_name='dev', value_name='value')
#  .assign(type_of_loss=t1.id)
)

,AY,1,2,3,4,5,6,7,8,9,10
0,1,357848,1124788.0,1735330.0,2218270.0,2745596.0,3319994.0,3466336.0,3606286.0,3833515.0,3901463.0
1,2,352118,1236139.0,2170033.0,3353322.0,3799067.0,4120063.0,4647867.0,4914039.0,5339085.0,NaN
2,3,290507,1292306.0,2218525.0,3235179.0,3985995.0,4132918.0,4628910.0,4909315.0,NaN,NaN
3,4,310608,1418858.0,2195047.0,3757447.0,4029929.0,4381982.0,4588268.0,NaN,NaN,NaN
4,5,443160,1136350.0,2128333.0,2897821.0,3402672.0,3873311.0,NaN,NaN,NaN,NaN
5,6,396132,1333217.0,2180715.0,2985752.0,3691712.0,NaN,NaN,NaN,NaN,NaN
6,7,440832,1288463.0,2419861.0,3483130.0,NaN,NaN,NaN,NaN,NaN,NaN
7,8,359480,1421128.0,2864498.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,376686,1363294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,344014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# type(rsv.tri.paid_loss)
rsv.tri.paid_loss.ata('vwa')

vol_wtd = {
                '5 Years': rsv.tri.paid_loss.ata('vwa', 5),
                '3 Years': rsv.tri.paid_loss.ata('vwa', 3),
                '2 Years': rsv.tri.paid_loss.ata('vwa', 2),

                # '5 Years': self.triangle.ata('vwa', 5),
                # '3 Years': self.triangle.ata('vwa', 3),
                # '2 Years': self.triangle.ata('vwa', 2),
            }
pd.DataFrame(vol_wtd).transpose()

,1,2,3,4,5,6,7,8,9,10
5 Years,3.244797,1.786666,1.468194,1.165122,1.103824,1.086269,1.053874,1.076555,1.017725,1.0
3 Years,3.460401,1.846507,1.392009,1.153852,1.084915,1.097355,1.053874,1.076555,1.017725,1.0
2 Years,3.782329,1.950242,1.406103,1.205795,1.110687,1.082476,1.058919,1.076555,1.017725,1.0


In [19]:
rsv.tri

"paid_loss"

In [20]:
vol_wtd  = pd.DataFrame({
    # 'vol wtd all years': rsv.tri.paid_loss.ata('vwa'),
    'Vol wtd': ["" for _ in range(10)],
    'all years': rsv.tri.paid_loss.ata('vwa').round(3)
    , '5 years': rsv.tri.paid_loss.ata('vwa', 5).round(3)
    , '3 years': rsv.tri.paid_loss.ata('vwa', 3).round(3)
    , '2 years': rsv.tri.paid_loss.ata('vwa', 2).round(3)
    }).transpose()

simple = pd.DataFrame({
    'Simple': ["" for _ in range(10)],
    'all years': rsv.tri.paid_loss.ata('simple').round(3),
    '5 years': rsv.tri.paid_loss.ata('simple', 5).round(3),
    '3 years': rsv.tri.paid_loss.ata('simple', 3).round(3),
    '2 years': rsv.tri.paid_loss.ata('simple', 2).round(3)}).transpose()

medial = pd.DataFrame({
    'Medial 5-Year': ["" for _ in range(10)],
    'ex hi/low': rsv.tri.paid_loss.ata('medial', 5, excludes='hl').round(3),
    'ex hi': rsv.tri.paid_loss.ata('medial', 5, excludes='h').round(3),
    'ex low': rsv.tri.paid_loss.ata('medial', 5, excludes='l').round(3),
                         }).transpose()

pd.concat([rsv.tri.paid_loss.ata().round(3).drop(index=10), vol_wtd, simple, medial], axis=0).drop(columns='10').fillna('')

,1,2,3,4,5,6,7,8,9
1,3.143,1.543,1.278,1.238,1.209,1.044,1.04,1.063,1.018
2,3.511,1.755,1.545,1.133,1.084,1.128,1.057,1.086,
3,4.448,1.717,1.458,1.232,1.037,1.12,1.061,,
4,4.568,1.547,1.712,1.073,1.087,1.047,,,
5,2.564,1.873,1.362,1.174,1.138,,,,
6,3.366,1.636,1.369,1.236,,,,,
7,2.923,1.878,1.439,,,,,,
8,3.953,2.016,,,,,,,
9,3.619,,,,,,,,
Vol wtd,,,,,,,,,


In [21]:
rsv.tri.paid_loss.ata_summary()

,1,2,3,4,5,6,7,8,9
1,3.143,1.543,1.278,1.238,1.209,1.044,1.04,1.063,1.018
2,3.511,1.755,1.545,1.133,1.084,1.128,1.057,1.086,
3,4.448,1.717,1.458,1.232,1.037,1.12,1.061,,
4,4.568,1.547,1.712,1.073,1.087,1.047,,,
5,2.564,1.873,1.362,1.174,1.138,,,,
6,3.366,1.636,1.369,1.236,,,,,
7,2.923,1.878,1.439,,,,,,
8,3.953,2.016,,,,,,,
9,3.619,,,,,,,,
Vol Wtd,,,,,,,,,


In [22]:
# melted = rsv.tri.paid_loss.melt_triangle()

# melted['AY'] = melted['AY'].astype(str).str.zfill(4).astype('category')
# melted['dev'] = melted['dev'].astype(str).str.zfill(4).astype('category')

# dm_total = pd.get_dummies(melted, columns=['AY', 'dev'], drop_first=True)
# dm_ay = pd.get_dummies(melted[['AY']], drop_first=True)
# dm_dev = pd.get_dummies(melted[['dev']], drop_first=True)

# #each row is a triangle cell, and is the cumulative sum of the previous AY and dev columns
# cols = dm_ay.columns.tolist()
# cols.reverse()
# for i, c in enumerate(cols):
#     if i==0:
#         dm_total[c] = dm_ay[c]
#     else:
#         dm_total[c] = dm_ay[c] + dm_total[cols[i-1]]

# cols = dm_dev.columns.tolist()
# cols.reverse()
# for i, c in enumerate(cols):
#     if i==0:
#         dm_total[c] = dm_dev[c]
#     else:
#         dm_total[c] = dm_dev[c] + dm_total[cols[i-1]]

# # dm_total.insert(0, 'AY_dev', dm_total.index)
 
# dm_total['paid_loss'].notnull().astype(int)

In [23]:
# dm = rsv.tri.paid_loss.base_design_matrix()
# X = dm.drop(columns='paid_loss')
# y = dm['paid_loss']

# X_train = X.loc[X.is_observed.eq(1)].rename(columns={'is_observed': 'intercecpt'})
# y_train = y[X.is_observed.eq(1)]

# X_forecast = X.loc[X.is_observed.eq(0)].rename(columns={'is_observed': 'intercecpt'}).assign(intercept=1)
# y_forecast = y[X.is_observed.eq(0)]

# X_train.head()

### incremental triangle

In [24]:
rsv.tri.paid_loss.cum_to_inc()
rsv.tri.paid_loss.incr_triangle

,1,2,3,4,5,6,7,8,9,10
AY,,,,,,,,,,
1,357848,766940.0,610542.0,482940.0,527326.0,574398.0,146342.0,139950.0,227229.0,67948.0
2,352118,884021.0,933894.0,1183289.0,445745.0,320996.0,527804.0,266172.0,425046.0,NaN
3,290507,1001799.0,926219.0,1016654.0,750816.0,146923.0,495992.0,280405.0,NaN,NaN
4,310608,1108250.0,776189.0,1562400.0,272482.0,352053.0,206286.0,NaN,NaN,NaN
5,443160,693190.0,991983.0,769488.0,504851.0,470639.0,NaN,NaN,NaN,NaN
6,396132,937085.0,847498.0,805037.0,705960.0,NaN,NaN,NaN,NaN,NaN
7,440832,847631.0,1131398.0,1063269.0,NaN,NaN,NaN,NaN,NaN,NaN
8,359480,1061648.0,1443370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,376686,986608.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### build design matrix `X` for a linear model
- the ay parameters and dev parameters are represented by trends, not levels

In [25]:
rsv.tri.paid_loss.base_linear_model()
X = rsv.tri.paid_loss.X_base_train
X.head()

,intercept,AY_0002,AY_0003,AY_0004,AY_0005,AY_0006,AY_0007,AY_0008,AY_0009,AY_0010,dev_0002,dev_0003,dev_0004,dev_0005,dev_0006,dev_0007,dev_0008,dev_0009,dev_0010
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### use most recent calendar period as the validation period

In [26]:
# cal periods
X_id = rsv.tri.paid_loss.X_id_train

X_train = X.loc[X_id.cal.ne(X_id.cal.max()), :]
X_val = X.loc[X_id.cal.eq(X_id.cal.max()), :]

print(f"X shape: {X.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")

X shape: (55, 19)
X_train shape: (45, 19)
X_val shape: (10, 19)


### target `y` is the incremental loss

In [27]:
y = rsv.tri.paid_loss.y_base_train
y_train = y[X_id.cal.ne(X_id.cal.max())]
y_val = y[X_id.cal.eq(X_id.cal.max())]

print(f"y shape: {y.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_val shape: {y_val.shape}")

y shape: (55,)
y_train shape: (45,)
y_val shape: (10,)


In [28]:
pd.concat([pd.DataFrame(dict(y=y)), X_id, X], axis=1).head()

,y,ay,dev,cal,intercept,AY_0002,AY_0003,AY_0004,AY_0005,AY_0006,...,AY_0010,dev_0002,dev_0003,dev_0004,dev_0005,dev_0006,dev_0007,dev_0008,dev_0009,dev_0010
0,357848.0,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,352118.0,2,1,2,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,290507.0,3,1,3,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,310608.0,4,1,4,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,443160.0,5,1,5,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0


### fit the model

In [29]:
from sklearn.linear_model import PoissonRegressor

model = PoissonRegressor(alpha=1, max_iter=1000)
model.fit(X_train, y_train)


PoissonRegressor(alpha=1, max_iter=1000)

In [30]:
parameter_table = pd.DataFrame({   
    'params':model.feature_names_in_
    ,'coef':[model.intercept_] + model.coef_
    })

parameter_table

,params,coef
0,intercept,12.554200
1,AY_0002,12.863926
2,AY_0003,12.543591
3,AY_0004,12.596011
4,AY_0005,12.393036
5,AY_0006,12.573752
6,AY_0007,12.723400
7,AY_0008,12.555323
8,AY_0009,12.472089
9,AY_0010,12.554519


In [31]:
testdf = pd.concat([pd.DataFrame(dict(y=y)), X_id, X], axis=1)
testdf['y_pred'] = model.predict(X)
testdf['raw_residual'] = testdf.y - testdf.y_pred

testdf = testdf['raw_residual y y_pred ay dev cal'.split() + parameter_table.params.tolist()]
testdf.head()

,raw_residual,y,y_pred,ay,dev,cal,intercept,AY_0002,AY_0003,AY_0004,...,AY_0010,dev_0002,dev_0003,dev_0004,dev_0005,dev_0006,dev_0007,dev_0008,dev_0009,dev_0010
0,74565.451915,357848.0,283282.548085,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,-33887.697975,352118.0,386005.697975,2,1,2,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,-91303.449871,290507.0,381810.449871,3,1,3,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-87377.959616,310608.0,397985.959616,4,1,4,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,104521.283161,443160.0,338638.716839,5,1,5,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [228]:
def actual_vs_predicted(df, observed_col, predicted_col, title='Observed vs Predicted', hover_data=['ay', 'dev', 'cal'], height=600, width=1000, trendline='ols'):
    #
    fig = px.scatter(df, x=observed_col, y=predicted_col, title=title,
                     hover_data=hover_data, height=height, width=width,
                     trendline=trendline)
    fig.add_shape(
        type="line",
        x0=0,
        y0=0,
        x1=df.y.max(),
        y1=df.y.max(),
        name='Line of Equality',
        line=dict(
            color="black",
            width=2,
            dash="dashdot",
        ),
    )

    fig.update_layout(
        legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        )
    )

    

    return fig

# fig = px.scatter(testdf, x='y', y='y_pred', title='y vs y_pred', color='ay', hover_data=['ay', 'dev', 'cal'], height=600, width=1000, trendline='ols')

# # add a 45 degree line
# fig.add_shape(
#     type="line",
#     x0=0,
#     y0=0,
#     x1=testdf.y.max(),
#     y1=testdf.y.max(),
#     line=dict(
#         color="black",
#         width=2,
#         dash="dashdot",
#     ),
# )

# fig.show()

actual_vs_predicted(testdf, 'y', 'y_pred', 'Observed vs Predicted', ['ay', 'dev', 'cal'], 600, 1000, 'ols')

In [33]:
def plot_standardized_pearson_residuals_vs_fitted(df,
                                                  model,
                                                  title='Standardized Pearson Residuals vs Fitted Values'
                                                  ) -> None:
    """
    Plots the standardized Pearson residuals versus the fitted values for a given dataset and model.
    
    Args:
    - df: Pandas DataFrame containing the input and output data (y and y_pred).
    - model: Trained regression model object with a 'predict' method that can make predictions on input data.
    - title: String representing the title of the plot (default is 'Standardized Pearson Residuals vs Fitted Values').
    
    Returns:
    - None. Displays a scatter plot of the standardized Pearson residuals versus the fitted values.
    """
    import numpy as np
    from sklearn.metrics import mean_squared_error
    
    # calculate standardized pearson residuals
    df['residuals'] = (df['y'] - df['y_pred']) / np.sqrt(mean_squared_error(df['y'], df['y_pred']))

    # create message for hover template
    hover  = '<b>Fitted Values:</b> %{x}<br>'
    hover += '<b>Standardized Pearson Residuals:</b> %{customdata[3]}<br>'
    hover += '<b>AY:</b> %{customdata[0]}<br>'
    hover += '<b>DY:</b> %{customdata[1]}<br>'
    hover += '<b>CY:</b> %{customdata[2]}<br>'
    hover += '<extra></extra>'
    
    # add a scatter plot of the standardized residuals vs the fitted values, and a horizontal line at 0
    fig = go.Figure(go.Scatter(x=df['y_pred'],
                            y=df['residuals'],
                            mode='markers',
                            customdata=(df[['ay', 'dev', 'cal', 'residuals']]
                                        .assign(residuals=df.residuals.round(3))).values,
                            marker=dict(color='red',
                                        colorscale='RdBu',
                                        line=dict(color='black', width=1),
                                        opacity=0.5),
                            hovertemplate=hover,
                            showlegend=False))

    fig.update_layout(title=title,
                    xaxis_title='Fitted Values',
                    yaxis_title='Standardized Pearson Residuals',
                    coloraxis_showscale=False)

    
    # add a horizontal line at 0
    fig.add_shape(
        type="line",
        x0=0,
        y0=0,
        x1=df['y_pred'].max(),
        y1=0,
        line=dict(
            color="black",
            width=3,
            dash="dashdot",
        ),
    )
    
    # show the plot
    fig.show()


plot_standardized_pearson_residuals_vs_fitted(testdf, model)

In [34]:
def plot_standardized_residuals_by_column(df, model, x_col, title=None) -> None:
    import numpy as np
    import plotly.graph_objects as go
    from sklearn.metrics import mean_squared_error
    
    if title is None:
        title = f'Standardized Pearson Residuals by {x_col}'
    
    # Calculate y_pred
    # df['y_pred'] = model.predict(df.drop('y', axis=1))
    
    # Calculate standardized Pearson residuals
    df['residuals'] = (df['y'] - df['y_pred']) / np.sqrt(mean_squared_error(df['y'], df['y_pred']))
    
    fig = go.Figure(go.Scatter(x=df[x_col],
                               y=df['residuals'],
                               mode='markers',
                               customdata=df[['ay', 'dev', 'cal']].values,
                               marker=dict(color=df['y'],
                                           colorscale='RdBu',
                                           line=dict(color='black', width=1),
                                           opacity=0.5),
                               hovertemplate=f'<b>{x_col}:</b> %{{x}}<br><b>Standardized Pearson Residuals:</b> %{{y}}<br><b>AY:</b> %{{customdata[0]}}<br><b>Dev:</b> %{{customdata[1]}}<br><b>Cal:</b> %{{customdata[2]}}<extra></extra>',
                               showlegend=False))
    
    fig.update_layout(title=title,
                      xaxis_title=x_col,
                      yaxis_title='Standardized Pearson Residuals',
                      coloraxis_showscale=False)
    
    fig.show()


# Example usage:
plot_standardized_residuals_by_column(testdf, model, 'ay')


In [35]:
def plot_standardized_residuals_by_column(df, model, x_col, title=None, return_=False) -> None:

    if title is None:
        title = f'Standardized Pearson Residuals by {x_col}'

    # Group by x_col and calculate the mean and standard deviation of residuals for each group
    grouped = df.groupby(x_col)['residuals'].agg(['mean', 'std']).reset_index()

    fig = go.Figure()

    # Scatter plot of residuals
    fig.add_trace(go.Scatter(x=df[x_col],
                             y=df['residuals'],
                             mode='markers',
                             customdata=df[['ay', 'dev', 'cal']].values,
                             marker=dict(color=df['y'],
                                         colorscale='RdBu',
                                         line=dict(color='black', width=1),
                                         opacity=0.5),
                             hovertemplate=f'<b>{x_col}:</b> %{{x}}<br><b>Standardized Pearson Residuals:</b> %{{y}}<br><b>AY:</b> %{{customdata[0]}}<br><b>Dev:</b> %{{customdata[1]}}<br><b>Cal:</b> %{{customdata[2]}}<extra></extra>',
                             showlegend=False))

    # Plot mean residuals
    fig.add_trace(go.Scatter(x=grouped[x_col],
                             y=grouped['mean'],
                             mode='lines',
                             line=dict(color='darkgray', width=2),
                             showlegend=False))

    # Plot standard deviation bounds and shading
    for level, fillcolor in zip([1, -1], ['rgba(128, 128, 128, 0.1)', 'rgba(128, 128, 128, 0.1)']):
        fig.add_trace(go.Scatter(x=grouped[x_col],
                                 y=grouped['mean'] + level * grouped['std'],
                                 mode='lines',
                                 line=dict(color='gray', width=1, dash="dot"),
                                 fill='tonexty',
                                 fillcolor=fillcolor,
                                 showlegend=False))

    fig.update_layout(title=title,
                      xaxis_title=x_col,
                      yaxis_title='Standardized Pearson Residuals',
                      coloraxis_showscale=False)

    fig.show()

    if return_:
        return fig

# Example usage:
x = plot_standardized_residuals_by_column(testdf, model, 'cal', return_=True)

In [36]:
%run plot.py

In [37]:
residual_plot(testdf, model, 'fitted')

In [187]:
# import pandas as pd
# import numpy as np
# import sklearn

# def pearson_residuals(df : pd.DataFrame
#                       , observed_col : str = 'y'
#                       , expected_col : str = 'y_pred'
#                       ) -> pd.Series:
#     """
#     Calculate Pearson residuals for the given DataFrame.

#     Parameters
#     ----------
#     df : pd.DataFrame
#         The input DataFrame containing the specified columns
#     observed_col : str, optional
#         The name of the column containing the observed values (default: 'y')
#     expected_col : str, optional
#         The name of the column containing the expected values (default: 'y_pred')

#     Returns
#     -------
#     pd.Series
#         A pandas Series containing the Pearson residuals
#     """
#     observed = df[observed_col]
#     expected = df[expected_col]

#     pearson_res = (observed - expected) / np.sqrt(expected)
#     return pearson_res

# def poisson_deviance_residuals(df : pd.DataFrame
#                                , y_col : str = 'y'
#                                , yhat_col : str = 'y_pred'
#                                ) -> pd.Series:
#     """
#     Calculate the deviance residuals for a Poisson GLM.
    
#     Parameters
#     ----------
#     df : pd.DataFrame
#         The input DataFrame containing 'y' and 'yhat' columns
#     y_col : str, optional
#         The name of the column containing the observed response variable (default: 'y')
#     yhat_col : str, optional
#         The name of the column containing the predicted response variable (default: 'yhat')
        
#     Returns
#     -------
#     pd.Series
#         A pandas Series containing the deviance residuals
#     """
#     y = df[y_col]
#     yhat = df[yhat_col]
    
#     # deviance residuals for a Poisson GLM are defined as:
#     # 2 * (y * log(y / yhat) - (y - yhat))
#     residuals = 2 * (y * np.log(y / yhat) - (y - yhat))
#     return pd.Series(np.where(y == 0, -2 * yhat, residuals), name='deviance_residuals')

# def calculate_hat_matrix_poisson(df : pd.DataFrame
#                                  , model : sklearn.linear_model.PoissonRegressor
#                                  , y_pred_col : str = 'y_pred'
#                                  , x_col : list = None
#                                  ) -> np.ndarray:
#     """
#     Calculate the hat matrix for the given DataFrame using a Poisson GLM.

#     Parameters
#     ----------
#     df : pd.DataFrame
#         The input DataFrame containing the specified columns
#     model : sklearn.linear_model.PoissonRegressor
#         The fitted PoissonRegressor model
#     y_pred_col : str, optional
#         The name of the column containing the expected values (default: 'y_pred')
#     x_col : list of str, optional
#         The list of column names used as predictors in the model
#     """

#     y_pred = df[y_pred_col].values

#     if x_col is None:
#         x_col = ['const'] + list(df.columns)

#     X = df[x_col].values
#     W = np.diag(y_pred)

#     # Compute the condition number of the matrix (X.T @ W @ X)
#     matrix_to_invert = X.T @ W @ X
#     cond_num = np.linalg.cond(matrix_to_invert)

#     # If the condition number is large, use the pseudoinverse
#     if cond_num > 1 / np.sqrt(np.finfo(matrix_to_invert.dtype).eps):
#         inverted_matrix = np.linalg.pinv(matrix_to_invert)
#     else:
#         inverted_matrix = np.linalg.inv(matrix_to_invert)

#     # Compute hat matrix for Poisson GLM
#     hat_matrix = np.sqrt(W) @ X @ inverted_matrix @ X.T @ np.sqrt(W)
#     return hat_matrix


def standardized_pearson_residuals_poisson(df, model, y_col='y', y_pred_col='y_pred', x_col=None):
    """
    Calculate standardized Pearson residuals for the given DataFrame using a Poisson GLM.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing the specified columns
    model : sklearn.linear_model.PoissonRegressor
        The fitted PoissonRegressor model
    y_col : str, optional
        The name of the column containing the observed values (default: 'y')
    y_pred_col : str, optional
        The name of the column containing the expected values (default: 'y_pred')
    x_col : list of str, optional
        The list of column names used as predictors in the model
    """
    
    hat_matrix = calculate_hat_matrix_poisson(df, model, y_pred_col=y_pred_col, x_col=x_col)
    
    y = df[y_col].values
    y_pred = df[y_pred_col].values
    # pearson_res = (y - y_pred) / np.sqrt(y_pred)

    h = np.diag(hat_matrix)
    standardized_res = pearson_res / np.where(np.logical_not(np.equal(h, 1)), np.sqrt(1 - h), np.nan)
    return standardized_res

def leverage_poisson(df : pd.DataFrame
                     , model : sklearn.linear_model.PoissonRegressor
                     , y_pred_col : str = 'y_pred'
                     , x_col : list = None
                     ) -> np.ndarray:
    """
    Calculate leverage values for the given DataFrame using a Poisson GLM.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing the specified columns
    model : sklearn.linear_model.PoissonRegressor
        The fitted PoissonRegressor model
    y_pred_col : str, optional
        The name of the column containing the expected values (default: 'y_pred')
    x_col : list of str, optional
        The list of column names used as predictors in the model
    """
    hat_matrix = calculate_hat_matrix_poisson(df, model, y_pred_col=y_pred_col, x_col=x_col)
    leverage_values = np.diag(hat_matrix)
    
    return leverage_values

def studentized_residuals_poisson(df, model, y_col='y', y_pred_col='y_pred', x_col=None):
    """
    Calculate studentized residuals for the given DataFrame using a Poisson GLM.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing the specified columns
    model : sklearn.linear_model.PoissonRegressor
        The fitted PoissonRegressor model
    y_col : str, optional
        The name of the column containing the observed values (default: 'y')
    y_pred_col : str, optional
        The name of the column containing the expected values (default: 'y_pred')
    x_col : list of str, optional
        The list of column names used as predictors in the model
    """
    
    hat_matrix = calculate_hat_matrix_poisson(df, model, y_pred_col=y_pred_col, x_col=x_col)
    
    y = df[y_col].values
    y_pred = df[y_pred_col].values
    pearson_res = (y - y_pred) / np.sqrt(y_pred)

    h = np.diag(hat_matrix)
    studentized_res = pearson_res / np.where(np.logical_not(np.equal(h, 1)), np.sqrt(1 - h), np.nan)
    return studentized_res



# testdf['Pearson Residuals'] = pearson_residuals(testdf)
testdf['Deviance Residuals'] = poisson_deviance_residuals(testdf)
testdf['Pearson Residuals'] = pearson_residuals(testdf)
testdf['Standardized Pearson Residuals'] = standardized_pearson_residuals_poisson(testdf, model, x_col=['intercept'] + testdf.columns[testdf.columns.str.contains("AY_") | testdf.columns.str.contains("dev_")].tolist())
testdf['Leverage'] = leverage_poisson(testdf, model, x_col=['intercept'] + testdf.columns[testdf.columns.str.contains("AY_") | testdf.columns.str.contains("dev_")].tolist())
testdf['Studentized Residuals'] = studentized_residuals_poisson(testdf, model, x_col=['intercept'] + testdf.columns[testdf.columns.str.contains("AY_") | testdf.columns.str.contains("dev_")].tolist())

testdf.head(10)

,raw_residual,y,y_pred,ay,dev,cal,intercept,AY_0002,AY_0003,AY_0004,...,dev_0007,dev_0008,dev_0009,dev_0010,residuals,Deviance Residuals,Standardized Pearson Residuals,Pearson Residuals,Leverage,Studentized Residuals
0,74565.451915,357848.0,283282.548085,1,1,1,1,0,0,0,...,0,0,0,0,0.485056,18101.137439,152.863504,140.096648,0.160061,152.863504
1,-33887.697975,352118.0,386005.697975,2,1,2,1,1,0,0,...,0,0,0,0,-0.220443,3066.118790,-60.653098,-54.543777,0.191306,-60.653098
2,-91303.449871,290507.0,381810.449871,3,1,3,1,1,1,0,...,0,0,0,0,-0.593938,23817.744026,-164.386251,-147.762177,0.192029,-164.386251
3,-87377.959616,310608.0,397985.959616,4,1,4,1,1,1,1,...,0,0,0,0,-0.568402,20765.753593,-154.784582,-138.505820,0.199280,-154.784582
4,104521.283161,443160.0,338638.716839,5,1,5,1,1,1,1,...,0,0,0,0,0.679922,29374.918184,199.354673,179.612446,0.188254,199.354673
5,50917.318773,396132.0,345214.681227,6,1,6,1,1,1,1,...,0,0,0,0,0.331222,7165.836720,96.718064,86.660442,0.197164,96.718064
6,32104.956025,440832.0,408727.043975,7,1,7,1,1,1,1,...,0,0,0,0,0.208846,2458.249873,57.159306,50.217535,0.228143,57.159306
7,-49575.824344,359480.0,409055.824344,8,1,8,1,1,1,1,...,0,0,0,0,-0.322496,6266.981986,-90.906385,-77.513733,0.272943,-90.906385
8,-3.688003,376686.0,376689.688003,9,1,9,1,1,1,1,...,0,0,0,0,-0.000024,0.000036,-0.007561,-0.006009,0.368463,-0.007561
9,-32675.688003,344014.0,376689.688003,10,1,10,1,1,1,1,...,0,0,0,0,-0.212559,2920.138050,NaN,-53.239366,1.000000,NaN


In [63]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from sklearn.linear_model import LinearRegression

def plot_by(df, y_col, x_col, accident_col='ay', development_col='dev', calendar_col='cal', include_ols_trendline=True):
    """
    Plot a specified quantity against another column in the DataFrame.

    Parameters
    ----------
    df : pd.DataFrame
        The input DataFrame containing the specified columns
    y_col : str
        The name of the column containing the quantity to be plotted on the y-axis
    x_col : str
        The name of the column to be plotted on the x-axis
    accident_col : str, optional
        The name of the column containing accident information (default: None)
    development_col : str, optional
        The name of the column containing development information (default: None)
    calendar_col : str, optional
        The name of the column containing calendar information (default: None)
    include_ols_trendline : bool, optional
        If True, include an OLS trendline for float columns (default: True)

    Returns
    -------
    plotly.graph_objs._figure.Figure
        A Plotly Figure containing the scatter plot with hover information and trendlines
    """
    # replace inf values with nan
    df[x_col] = df[x_col].replace([np.inf, -np.inf], np.nan)
    df[y_col] = df[y_col].replace([np.inf, -np.inf], np.nan)

    hover_data = []
    if accident_col:
        hover_data.append(accident_col)
    if development_col:
        hover_data.append(development_col)
    if calendar_col:
        hover_data.append(calendar_col)

    fig = px.scatter(df, x=x_col, y=y_col, hover_data=hover_data)
    fig.update_layout(title=f'{y_col} by {x_col}', xaxis_title=x_col, yaxis_title=y_col)

    x_dtype = df[x_col].dtype

    if include_ols_trendline and np.issubdtype(x_dtype, np.floating):
        df_clean = df.dropna(subset=[x_col, y_col])

        X = df_clean[x_col].values.reshape(-1, 1)
        y = df_clean[y_col].values.reshape(-1, 1)
        ols = LinearRegression().fit(X, y)
        y_pred = ols.predict(X)

        fig.add_trace(go.Scatter(x=df_clean[x_col], y=y_pred[:, 0], mode='lines', name='OLS Trendline'))

    elif np.issubdtype(x_dtype, (np.integer, np.object)) or x_dtype.name == 'category':
        group_mean_std = df.groupby(x_col)[y_col].agg(['mean', 'std'])

        mean_line = go.Scatter(x=group_mean_std.index, y=group_mean_std['mean'], mode='lines', line=dict(dash='dot', width=3, color='black'), name='Mean')
        std_upper = go.Scatter(x=group_mean_std.index, y=group_mean_std['mean'] + group_mean_std['std'], mode='lines', line=dict(dash='dash', width=1, color='black'), name='Mean + 1 Std')
        std_lower = go.Scatter(x=group_mean_std.index, y=group_mean_std['mean'] - group_mean_std['std'], mode='lines', line=dict(dash='dash', width=1, color='black'), name='Mean - 1 Std', showlegend=False)

        fig.add_traces([mean_line, std_upper, std_lower])

    return fig

In [64]:
plot_by(testdf, 'Deviance Residuals', 'y_pred')

In [65]:
plot_by(testdf, 'Pearson Residuals', 'y_pred')

In [66]:
plot_by(testdf, 'Standardized Pearson Residuals', 'y_pred')

In [67]:
plot_by(testdf, 'Leverage', 'y_pred')

In [188]:
plot_by(testdf, 'Studentized Residuals', 'y_pred')

In [191]:
def qq_plot(df : pd.DataFrame
           , residual_col : str
           , title : str = None
           , acc_col : str = None
           , dev_col : str = None
           , cal_col : str = None
           , dist='norm'
           ) -> go.Figure:
    
    if acc_col is None:
        acc_col = 'ay'
    if dev_col is None:
        dev_col = 'dev'
    if cal_col is None:
        cal_col = 'cal'


    residuals = df[residual_col]

    # actual quantiles of the residuals
    q1 = residuals.sort_values().reset_index(drop=True)

    # theoretical quantiles
    if dist == 'norm':
        # normal distribution
        q2 = scipy.stats.norm.ppf(np.linspace(0.01, 0.99, residuals.shape[0]), loc=residuals.mean(), scale=residuals.std())
    elif dist == 't':
        # t distribution
        q2 = scipy.stats.t.ppf(np.linspace(0.01, 0.99, residuals.shape[0]), df=residuals.shape[0]-2, loc=residuals.mean(), scale=residuals.std())
    else:
        raise ValueError(f'Invalid distribution: {dist}. Must be "norm" or "t".')

    # plotly qq plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=q2, y=q1, mode='markers', name='Observed'))
    fig.update_traces(mode='markers', marker=dict(size=8, opacity=0.5))
    fig.update_layout(title=title or f'QQ Plot of {residuals.name}',
                      xaxis_title='Theoretical Quantiles',
                      yaxis_title='Observed Quantiles')
    
    # add customdata to hover
    fig.update_traces(customdata=df[[acc_col, dev_col, cal_col]])
    
    # add hover text, using customdata
    fig.update_traces(hovertemplate='<b>Observed: </b> %{y:.2f}<br><b>Theoretical: </b> %{x:.2f} <br><b>Acc: </b> %{customdata[0]}<br><b>Dev: </b> %{customdata[1]}<br><b>Cal: </b> %{customdata[2]}')
    
    # add line of equality
    fig.add_trace(go.Scatter(x=q2, y=q2, mode='lines', name='Expected', line=dict(color='black', width=3, dash='dashdot')))

    fig.show()

qq_plot(testdf, 'Pearson Residuals')
# sm.stats.descriptivestats.describe(testdf['Pearson Residuals']).round

In [130]:
qq_plot(testdf.loc[testdf['Deviance Residuals'].ne(0)], 'Deviance Residuals')

In [192]:
qq_plot(testdf.loc[testdf['Studentized Residuals'].ne(0)], 'Studentized Residuals', dist='t')

In [194]:
## histogram of residuals
def residual_hist(df, residual_col, title=None, acc_col=None, dev_col=None, cal_col=None, bins=None):
    if acc_col is None:
        acc_col = 'ay'
    if dev_col is None:
        dev_col = 'dev'
    if cal_col is None:
        cal_col = 'cal'
    if bins is None:
        bins = None
    
    fig = px.histogram(df, x=residual_col, marginal='box', hover_data=[acc_col, dev_col, cal_col], nbins=bins)
    fig.update_layout(title=title or f'Histogram of {residual_col}')
    fig.show()

residual_hist(testdf, 'Pearson Residuals', bins=25)

In [195]:
residual_hist(testdf, 'Deviance Residuals', bins=25)

In [196]:
# function to calculate the RMSE
def rmse(df, y_col='y', y_pred_col='y_pred') -> float:
    return np.sqrt(sklearn.metrics.mean_squared_error(df[y_col], df[y_pred_col]))

rmse(testdf)

153725.5098169908

In [197]:
#mean_squared_error, mean_absolute_error, mean_poisson_deviance, median_absolute_error, d2_tweedie_score

In [198]:
# function to calculate the MAE
def mae(df, y_col='y', y_pred_col='y_pred') -> float:
    return sklearn.metrics.mean_absolute_error(df[y_col], df[y_pred_col])

mae(testdf)

115724.84137346964

In [199]:
def mpd(df, y_col='y', y_pred_col='y_pred') -> float:
    return sklearn.metrics.mean_poisson_deviance(df[y_col], df[y_pred_col])

mpd(testdf)

41334.76739369808

In [200]:
def d2_tweedie(df, y_col='y', y_pred_col='y_pred') -> float:
    return sklearn.metrics.d2_tweedie_score(df[y_col], df[y_pred_col])

d2_tweedie(testdf)

0.8030158902826094

In [201]:
def log_likelihood(df, y_col='y', y_pred_col='y_pred') -> float:
    """
    Calculate the ln(likelihood) of the Poisson GLM model. Likelihood is the product of the
    Poisson PMF for each observation, so we take the log of the product to get the log-likelihood.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the actual and predicted values of the response variable.
    y_col : str, optional
        Name of the column containing the actual values of the response variable, by default 'y'
    y_pred_col : str, optional
        Name of the column containing the predicted values of the response variable, by default 'y_pred'

    Returns
    -------
    float
        The likelihood of the Poisson GLM model.
    """
    return np.sum(np.log(df[y_pred_col]) - df[y_col] * np.log(df[y_pred_col]) + np.log(df[y_pred_col]))
    

log_likelihood(testdf)

-462172251.7244816

In [202]:
def aic(df, y_col='y', y_pred_col='y_pred', k=2) -> float:
    """
    Calculate the AIC of the Poisson GLM model.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the actual and predicted values of the response variable.
    y_col : str, optional
        Name of the column containing the actual values of the response variable, by default 'y'
    y_pred_col : str, optional
        Name of the column containing the predicted values of the response variable, by default 'y_pred'
    k : int, optional
        Number of parameters in the model, by default 2

    Returns
    -------
    float
        The AIC of the Poisson GLM model.
    """
    return -2 * log_likelihood(df, y_col, y_pred_col) + 2 * k

aic(testdf, k=parameter_table.params.nunique())

924344541.4489632

In [203]:
def bic(df, y_col='y', y_pred_col='y_pred', k=2) -> float:
    """
    Calculate the BIC of the Poisson GLM model.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the actual and predicted values of the response variable.
    y_col : str, optional
        Name of the column containing the actual values of the response variable, by default 'y'
    y_pred_col : str, optional
        Name of the column containing the predicted values of the response variable, by default 'y_pred'
    k : int, optional
        Number of parameters in the model, by default 2

    Returns
    -------
    float
        The BIC of the Poisson GLM model.
    """
    return -2 * log_likelihood(df, y_col, y_pred_col) + np.log(df.shape[0]) * k

bic(testdf, k=parameter_table.params.nunique())

924344579.5882937

In [237]:
def cooks_distance(df, x_col=None, y_pred_col='y_pred', model=None, boolean=False) -> float:
    """
    Calculate the Cook's distance of the Poisson GLM model. Cook's distance is a measure of the
    influence of an observation on the model. It is calculated as the squared residuals
    multiplied by the leverage of the observation, divided by the number of parameters in the
    model and the (1 - leverage) of the observation.

    Higher values of Cook's distance indicate that the observation has a greater influence on
    the model, and a value is considered influential if it is greater than the median of an F   
    distribution with p and n-p-1 degrees of freedom, where p is the number of parameters in the
    model and n is the number of observations.

    Cook's distance can also expressed using the leverage and studentized residuals:

    $$
    \text{Cook's distance} = \frac{t_i^2}{p} \cdot \frac{h_{ii}}{1-h_{ii}}
    $$

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the actual and predicted values of the response variable.
    x_col : str, optional
        Name of the column containing the explanatory variables, by default None
    y_pred_col : str, optional
        Name of the column containing the predicted values of the response variable, by default 'y_pred'
    model : sklearn.linear_model.PoissonRegressor, optional
        The Poisson GLM model, by default None
    boolean : bool, optional
        Whether to return a boolean array of observations with a Cook's distance greater than the
        cutoff, by default False

    Returns
    -------
    float
        The Cook's distance of the Poisson GLM model.

    """
    n_params = model.coef_.shape[0] + 1
    leverage = leverage_poisson(df=df, model=model, y_pred_col=y_pred_col, x_col=x_col)
    studentized_residuals = studentized_residuals_poisson(df=df, model=model, y_pred_col=y_pred_col, x_col=x_col)
    cooks_d = (studentized_residuals ** 2) * leverage / (n_params * (1 - leverage))
    cooks_d = pd.Series(cooks_d, index=df.index, name="Cook's Distance")

    if boolean:
        cutoff = scipy.stats.f.median(n_params, df.shape[0] - n_params - 1)
        return cooks_d.fillna(0) > cutoff
    else:
        return cooks_d

x_cols = rsv.tri.paid_loss.X_base.columns[1:].tolist() + ['intercept']
cooks_distance(testdf, x_col=x_cols, model=model)[:15].round(1)
    

C:\Users\AndyW\AppData\Local\Temp\ipykernel_15928\1949150738.py:181: RuntimeWarning:

invalid value encountered in sqrt



0     222.6
1      43.5
2     321.1
3     298.1
4     460.8
5     114.9
6      48.3
7     155.1
8       0.0
9       NaN
10    158.2
11    164.0
12    109.8
13    526.1
14    842.1
Name: Cook's Distance, dtype: float64

In [243]:
dat = testdf.copy()
dat = dat.loc[np.logical_not(dat.ay.eq(1) & dat.dev.eq(10))]
dat = dat.loc[np.logical_not(dat.ay.eq(10) & dat.dev.eq(1))]
dat['cooks_d'] = cooks_distance(dat, x_col=x_cols, model=model)
dat['cooks_ratio'] = dat['cooks_d'] / dat['y']
px.scatter(dat, x='y', y='cooks_ratio', trendline='ols', hover_data=['ay', 'dev', 'cal', 'cooks_d'])

In [230]:
def loo_cv(df, y_col='y', y_pred_col='y_pred', x_cols=None) -> float:
    """
    Calculate the leave-one-out cross-validation (LOO-CV) of the Poisson GLM model.

    Leave-one-out cross-validation is a method of estimating the out-of-sample prediction error
    of a model by removing one observation at a time. The LOO-CV is calculated as the sum of the squared 
    errors of the holdout observations, divided by the number of observations.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the actual and predicted values of the response variable.
    y_col : str, optional
        Name of the column containing the actual values of the response variable, by default 'y'
    y_pred_col : str, optional
        Name of the column containing the predicted values of the response variable, by default 'y_pred'
    x_cols : list, optional
        List of column names containing the explanatory variables, by default None

    Returns
    -------
    float
        The LOO of the Poisson GLM model.
    """
    X = df[x_cols]
    y = df[y_col]

    loo_scores = []

    skipped = 0
    loo = LeaveOneOut()
    for train, test in loo.split(X):
        if len(test) == 0:
            skipped += 1
        elif y.iloc[test].values[0] == 0:
            skipped += 1
        else:
            X_train, X_test = X.iloc[train], X.iloc[test]
            y_train, y_test = y.iloc[train], y.iloc[test]
            model = PoissonRegressor().fit(X_train, y_train)
            y_pred = model.predict(X_test)
            loo_scores.append((y_test - y_pred) ** 2)

    return np.sum(loo_scores) / (df.shape[0] - skipped)

X = rsv.X_train.copy()
X = X.loc[:, (X.columns.str.startswith('AY_') | X.columns.str.startswith('dev_'))]
X['intercept'] = 1

x_cols = X.columns.tolist()
loo_cv(testdf, x_cols=x_cols)
# X.columns

AttributeError: 'ROCKY' object has no attribute 'X_train'

In [ ]:
def hdim():
    """
    Calculate the high-dimensional influence measure (HDIM) of the Poisson GLM model. HDIM is a
    measure of the influence of an observation on the model. Instead of typical calculations based
    on regression residuals, HDIM uses marginal correlations between the response variable and

    References
    ----------
    Junlong Zhao, Chenlei Leng, Lexin Li, Hansheng Wang. "High-dimensional influence measure." The Annals
    of Statistics, 41(5) 2639-2667 October 2013.
    
    https://doi.org/10.1214/13-AOS1165
    
    Abstract 

    Influence diagnosis is important since presence of influential observations could lead to distorted analysis and
    misleading interpretations. For high-dimensional data, it is particularly so, as the increased dimensionality and
    complexity may amplify both the chance of an observation being influential, and its potential impact on the
    analysis. In this article, we propose a novel high-dimensional influence measure for regressions with the
    number of predictors far exceeding the sample size. Our proposal can be viewed as a high-dimensional
    counterpart to the classical Cook’s distance. However, whereas the Cook’s distance quantifies the individual
    observation’s influence on the least squares regression coefficient estimate, our new diagnosis measure
    captures the influence on the marginal correlations, which in turn exerts serious influence on downstream
    analysis including coefficient estimation, variable selection and screening. Moreover, we establish the
    asymptotic distribution of the proposed influence measure by letting the predictor dimension go to infinity.
    Availability of this asymptotic distribution leads to a principled rule to determine the critical value for
    influential observation detection. Both simulations and real data analysis demonstrate usefulness of the new
    influence diagnosis measure.
    """


In [236]:
def vif(df, x_cols=None):
    """
    Calculate the variance inflation factor (VIF) of the Poisson GLM model. VIF is a measure of the
    inflation of the variance of the coefficient estimates due to collinearity.

    VIF estimates larger than 10 indicate high multicollinearity between this explanatory variable
    and the others, and the parameter estimates will have large standard errors because of this.

    Parameters
    ----------
    df : pandas.DataFrame
        DataFrame containing the explanatory variables.
    x_cols : list, optional
        List of column names containing the explanatory variables, by default None

    Returns
    -------
    float
        The VIF of the Poisson GLM model.
    """
    X = df[x_cols]
    vifs = []
    col_names = []
    for i in range(X.shape[1]):
        y = X.iloc[:, i]
        X_ = X.drop(X.columns[i], axis=1)
        model = PoissonRegressor().fit(X_, y)
        vifs.append(1 / (1 - model.score(X_, y)))
        col_names.append(X.columns[i])
    return pd.Series(vifs, index=col_names, name='VIF Statistic')

vif(rsv.tri.paid_loss.X_base_train.drop(columns='intercept'),
    x_cols=rsv.tri.paid_loss.X_base_train.drop(columns='intercept').columns.tolist())

AY_0002     1.128425
AY_0003     1.232484
AY_0004     1.299781
AY_0005     1.317464
AY_0006     1.287887
AY_0007     1.224691
AY_0008     1.147508
AY_0009     1.075684
AY_0010     1.023976
dev_0002    1.128425
dev_0003    1.232484
dev_0004    1.299781
dev_0005    1.317464
dev_0006    1.287887
dev_0007    1.224691
dev_0008    1.147508
dev_0009    1.075684
dev_0010    1.023976
Name: VIF Statistic, dtype: float64